In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tslearn.generators import random_walks
import sklearn.gaussian_process as sklgp
import matplotlib.pyplot as plt
import numpy as np
from tsfeatures import tsfeatures
from tslearn.clustering import TimeSeriesKMeans
from datetime import date,datetime
from dateutil.parser import parse
from tsfeatures import acf_features
from tsfeatures import stl_features
from tsfeatures import entropy
from tsfeatures import frequency
from tqdm import tqdm
from sklearn.cluster import KMeans
from yellowbrick.cluster.elbow import kelbow_visualizer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib
from scipy.stats import skew
from sklearn.model_selection import cross_val_score
from itertools import product

In [ ]:
# 데이터 불러오기
data = pd.read_csv('./Innovine(1212).csv')

In [ ]:
data.columns
data=data.astype({'Date':'str'})
data['Date']=pd.to_datetime(data["Date"])
print(data.dtypes)

Unnamed: 0                     int64
Date                  datetime64[ns]
month_sin                    float64
month_cos                    float64
day_sin                      float64
                           ...      
yesterdayweek_cos            float64
yesterdayweek_sin            float64
diff_max_today_sum             int64
diff_min_today_sum             int64
diff_max_min_sum               int64
Length: 78, dtype: object


In [ ]:
data.columns
data=data.drop(['Unnamed: 0'],axis='columns')

In [ ]:
# 49개의 ProdCode에 해당하는 행 추출
ProdCode_select = ((data['ProdCode']=='BSK-BGR-001')| (data['ProdCode']=='BSK-BGR-003')| (data['ProdCode']=='BSK-BGR-004')|
            (data['ProdCode']=='BSK-BGR-005')| (data['ProdCode']=='BSK-CDG-001')| (data['ProdCode']=='BSK-CDG-002')|
            (data['ProdCode']=='BSK-CDG-003')| (data['ProdCode']=='BSK-CDG-004')| (data['ProdCode']=='BSK-CHK-002')|
            (data['ProdCode']=='BSK-CHK-003')| (data['ProdCode']=='BSK-GRN-001')| (data['ProdCode']=='BSK-GRN-002')|
            (data['ProdCode']=='BSK-GRN-003')| (data['ProdCode']=='BSK-GRN-004')| (data['ProdCode']=='BSK-GRN-005')|
            (data['ProdCode']=='BSK-GRN-006')| (data['ProdCode']=='BSK-GRN-007')| (data['ProdCode']=='BSK-GRN-008')|
            (data['ProdCode']=='BSK-HBR-001')| (data['ProdCode']=='BSK-HBR=002')| (data['ProdCode']=='BSK-HBR-003')|
            (data['ProdCode']=='BSK-KMB-001')| (data['ProdCode']=='BSK-KMB-002')| (data['ProdCode']=='BSK-KMB-003')|
            (data['ProdCode']=='BSK-KMB-004')| (data['ProdCode']=='BSK-KMB-005')| (data['ProdCode']=='BSK-OMK-001')|
            (data['ProdCode']=='BSK-PKT-001')| (data['ProdCode']=='BSK-RMN-001')| (data['ProdCode']=='BSK-RMN-002')|
            (data['ProdCode']=='BSK-RMN-003')| (data['ProdCode']=='BSK-RMN-004')| (data['ProdCode']=='BSK-RMN-005')|
            (data['ProdCode']=='BSK-RMN-006')| (data['ProdCode']=='BSK-RMN-007')| (data['ProdCode']=='BSK-RMN-008')|
            (data['ProdCode']=='BSK-TPK-001')| (data['ProdCode']=='CASSAVA-0001')| (data['ProdCode']=='JSK-JJM-001')|
            (data['ProdCode']=='JSK-JSK-001')| (data['ProdCode']=='JSK-JSK-002')| (data['ProdCode']=='MIN-001')|
            (data['ProdCode']=='MIN-002')| (data['ProdCode']=='MIN-005')| (data['ProdCode']=='MIN-010')|
            (data['ProdCode']=='MIN-011')| (data['ProdCode']=='MIN-012')| (data['ProdCode']=='MIN-013')|
            (data['ProdCode']=='MIN-014'))


data = data[ProdCode_select]
data

,Date,month_sin,month_cos,day_sin,day_cos,ProdCode,CatName,Category_1,Category_2,TransQty_sum,...,nextweek_cos,nextweek_sin,holiday_sum_max,holiday_sum_min,yesterdayweek,yesterdayweek_cos,yesterdayweek_sin,diff_max_today_sum,diff_min_today_sum,diff_max_min_sum
0,2022-03-07,1.0,6.123234e-17,-2.449294e-16,1.000000,BSK-KMB-004,KIMBAB (DAGING SAPI),Food,Bunshik,26,...,0.623490,7.818315e-01,26,26,7,1.000000,-2.449294e-16,9,21,30
1,2022-03-08,1.0,6.123234e-17,7.818315e-01,0.623490,BSK-KMB-004,KIMBAB (DAGING SAPI),Food,Bunshik,34,...,-0.222521,9.749279e-01,34,26,1,0.623490,7.818315e-01,1,29,30
2,2022-03-09,1.0,6.123234e-17,9.749279e-01,-0.222521,BSK-KMB-004,KIMBAB (DAGING SAPI),Food,Bunshik,25,...,-0.900969,4.338837e-01,34,25,2,-0.222521,9.749279e-01,10,20,30
3,2022-03-10,1.0,6.123234e-17,4.338837e-01,-0.900969,BSK-KMB-004,KIMBAB (DAGING SAPI),Food,Bunshik,29,...,-0.900969,-4.338837e-01,34,25,3,-0.900969,4.338837e-01,6,24,30
4,2022-03-11,1.0,6.123234e-17,-4.338837e-01,-0.900969,BSK-KMB-004,KIMBAB (DAGING SAPI),Food,Bunshik,42,...,-0.222521,-9.749279e-01,42,25,4,-0.900969,-4.338837e-01,0,37,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,2022-09-11,-1.0,-1.836970e-16,-4.338837e-01,-0.900969,JSK-JSK-001,TANGSUYUK,Food,Chinese,50,...,-0.900969,4.338837e-01,57,11,2,-0.222521,9.749279e-01,17,41,58
9236,2022-09-12,-1.0,-1.836970e-16,-9.749279e-01,-0.222521,JSK-JSK-001,TANGSUYUK,Food,Chinese,21,...,-0.900969,-4.338837e-01,57,11,3,-0.900969,4.338837e-01,46,12,58
9237,2022-09-13,-1.0,-1.836970e-16,-7.818315e-01,0.623490,JSK-JSK-001,TANGSUYUK,Food,Chinese,10,...,-0.222521,-9.749279e-01,57,10,4,-0.900969,-4.338837e-01,57,1,58
9238,2022-09-14,-1.0,-1.836970e-16,-4.898587e-16,1.000000,JSK-JSK-001,TANGSUYUK,Food,Chinese,17,...,0.623490,-7.818315e-01,57,10,5,-0.222521,-9.749279e-01,50,8,58


In [ ]:
# Kmeans 함수

def kmeans_total(data,k_number,index):


    # train data
    traindata = data[data['Date']<'2022-08-13']
    train_test_data = data[data['Date']<='2022-08-13']
    # K-means

    # 수요 패턴 추출
    data_pattern = traindata.iloc[:,[5,0,(30+index)]]
    data_pattern.columns = ['unique_id','ds','y']
    data_pattern['unique_id']=data_pattern['unique_id'].astype('category')
    features = tsfeatures(data_pattern, freq=1)

    items = data_pattern.unique_id.unique()

    #Compute ADI and CV
    adi = []
    cv = []
    for i in tqdm(items):
        sold = data_pattern.loc[data_pattern.unique_id == i].sort_values(by=['ds'])
        interval = []

        cnt = 1
        for dem in sold.y:
            if dem != 0:
                interval.append(cnt)
                cnt = 1
            else:
                cnt+=1

        adi.append(np.sum(interval) / len(interval))
        cv.append(np.std(sold.y) / np.mean(sold.y))

    features['adi'] = adi
    features['cv'] = cv


    features.index = data_pattern.unique_id.unique()
    features = features.drop(['unique_id'],axis=1)
    scaler = MinMaxScaler()
    scaler.fit(features)
    scaled_features = pd.DataFrame(scaler.transform(features))

    scaled_features = scaled_features.reset_index()
    c = ['ProdCode']
    c.extend(features.columns)
    scaled_features.columns = c

    scaled_features = scaled_features.dropna(axis=1)

    scaled_features = scaled_features.set_index(keys="ProdCode")



    model = KMeans(n_clusters = k_number, random_state = 10)
    model.fit(scaled_features)

    scaled_features['cluster'] = model.fit_predict(scaled_features)
    scaled_features['ProdCode'] = features.index



    # Cluster별 ProdCode 저장
    list1 = scaled_features['ProdCode'][scaled_features['cluster']==0]
    list2 = scaled_features['ProdCode'][scaled_features['cluster']==1]
    list3 = scaled_features['ProdCode'][scaled_features['cluster']==2]
    list_1 = list(list1)
    list_2 = list(list2)
    list_3 = list(list3)


    train_test_data['cluster'] =0
    train_test_data=train_test_data.reset_index()
    train_test_data = train_test_data.drop(labels='index',axis=1)

    for i in range(0,len(train_test_data)):
        if train_test_data['ProdCode'][i] in list_1:
            train_test_data['cluster'][i] = 1
        elif train_test_data['ProdCode'][i] in list_2:
            train_test_data['cluster'][i] = 2
        else :
            train_test_data['cluster'][i] = 3

    cluster1_data = train_test_data[train_test_data['cluster']==1]
    cluster2_data = train_test_data[train_test_data['cluster']==2]
    cluster3_data = train_test_data[train_test_data['cluster']==3]

    cluster1_data = cluster1_data.drop(labels='cluster',axis=1)
    cluster2_data = cluster2_data.drop(labels='cluster',axis=1)
    cluster3_data = cluster3_data.drop(labels='cluster',axis=1)

    return cluster1_data,cluster2_data,cluster3_data

In [ ]:
# 스케일링

def scale_col(data):
    data_scale = data
    data_scale_colnames = data_scale.iloc[:,[1,2,3,4,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76]].columns


    # 정규화 진행 - 각 열의 최소값을 0, 최대값을 1로 (range 정규화)
    scaler = MinMaxScaler()
    data_scale2 = scaler.fit_transform(data_scale.iloc[:,[1,2,3,4,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76]])

    data_scale2=pd.DataFrame(data_scale2)
    data_scale2.columns=data_scale_colnames
    return data_scale2

# scale_col 함수를 cluster 3개에 대하여 동시에 실행

def scale_cluster3(data_1, data_2, data_3):
    data_scale_1 = scale_col(data_1)
    data_scale_2 = scale_col(data_2)
    data_scale_3 = scale_col(data_3)
    return data_scale_1, data_scale_2, data_scale_3

# scale한 열과 sales, prodcode, date 열을 합침

def total_scale(original_data,data,index):
    total_scale = pd.concat([original_data.iloc[:,0],original_data.iloc[:,5],original_data.iloc[:,(30+index)]],axis=1)
    total_scale = total_scale.reset_index()
    data = data.reset_index()
    total_scale = total_scale.drop(['index'],axis=1)
    data = data.drop(['index'],axis=1)
    total_scale = pd.concat([total_scale,data],axis=1)
    return total_scale

# total_scale 함수를 cluster 3개에 대하여 동시에 진행

def total_scale_cluster3(original_data_1,original_data_2,original_data_3,data_1, data_2, data_3,index):
    total_scale_1 = total_scale(original_data_1,data_1,index)
    total_scale_2 = total_scale(original_data_2,data_2,index)
    total_scale_3 = total_scale(original_data_3,data_3,index)
    return total_scale_1,total_scale_2,total_scale_3



In [ ]:
def MAPE_plus(y_test, y_pred):
    return np.mean(np.abs(((y_test+0.01) - (y_pred+0.01)) / (y_test+0.01))) * 100

def gaus(data,cluster_number):
    # Gaussian Process Regression
    X_train=data[data['Date']<'2022-08-13']
    y_train=data[data['Date']<'2022-08-13']
    X_train=X_train.iloc[:,3:]
    y_train=y_train.iloc[:,2]


    kernel = 1*RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
    gpr = GaussianProcessRegressor(kernel, n_restarts_optimizer=9, random_state=0)
    gpr.fit(X_train, y_train)
    # prediction
    X_test = data[data['Date']=='2022-08-13']
    X_test = X_test.iloc[:,3:]
    y_test = data[data['Date']=='2022-08-13']
    y_test = y_test.iloc[:,2]
    y_pred_mean, y_pred_std = gpr.predict(X_test, return_std=True)
    MSE = mean_squared_error(y_test, y_pred_mean)
    RMSE = np.sqrt(MSE)
    MAE = mean_absolute_error(y_test, y_pred_mean)
    MAPE = MAPE_plus(y_test, y_pred_mean)
    NRMSE = np.sqrt(mean_squared_error(y_test, y_pred_mean)) / (np.max(y_test) - np.min(y_test))
    return {cluster_number:[['RMSE = ',RMSE],['MAE = ',MAE],['MAPE = ',MAPE]]}, {cluster_number:[['y_actual = ',y_test],['y_pred = ', y_pred_mean]]}



In [ ]:
# ElasticNet
def elastic_net_cluster(data,index):
    # 모델 정의
    modelElasticNet = ElasticNet()
    # train feature들
    x_train = data.iloc[:,[1,2,3,4,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76]]
    # train시 맞출 feature (target variable) - w.1
    y_train = data.iloc[:,[30+index]]

    # grid search를 위해 0.001간격으로 0부터 1까지의 값을 저장
    # test를 위해 0.1간격으로 진행
    for i in range(0, 11, 1):
        i_1000 = (i/10.0)
        if i == 0:
            list_grid = [i_1000]
        else:
            list_grid.append(i_1000)

    # grid search

    # alpha와 l1_ratio에 각각 0.001간격으로 0부터 1까지의 값을 저장
    # test를 위해 0.1간격으로 진행

    alphas = list_grid
    l1_ratios = list_grid


    # 0.001간격으로 0부터 1까지의 값인 alpha와 l1_ratio를 각 쌍으로 묶어 그리드서피 진행
    # 예 - (alpha,l1_ratio) - (0,0), (0,0.001), ..., (0.001,0), ..., (1,1)

    try_num = 0
    for (alpha,l1_ratio) in product(alphas,l1_ratios):

        rmse,coef_num = elastic_net_feature_selection_process(x_train,y_train,alpha,l1_ratio)
        if try_num == 0:

            alpha_result = list()
            ratio_result = list()
            rmse_result = list()
            coef_num_result = list()

            alpha_result.append(alpha)
            ratio_result.append(l1_ratio)
            rmse_result.append(rmse)
            coef_num_result.append(coef_num)
        else:
            alpha_result.append(alpha)
            ratio_result.append(l1_ratio)
            rmse_result.append(rmse)
            coef_num_result.append(coef_num)
        try_num = try_num + 1

    # 각 alpha와 l1_ratio 값들에 대한 결과(rmse, 변수 선택 수)를 data frame으로 보여줌

    grid_search_result = {'alpha':alpha_result,'l1_ratio':ratio_result,'rmse':rmse_result,'coef_num':coef_num_result}
    grid_search_result_data_frame = pd.DataFrame(grid_search_result)
    print(grid_search_result)
    print(grid_search_result_data_frame)

    # rmse와 변수 선택 수를 rank 매겨 rmse는 700등 안이면서 변수 선택 개수는 50등 안에 있는 경우를 뽑음

    grid_search_result_data_frame['rank_rmse'] = grid_search_result_data_frame['rmse'].rank()
    grid_search_result_data_frame['rank_coef_num'] = grid_search_result_data_frame['coef_num'].rank()

    # 앞서 뽑은 경우들 중 rmse가 가장 작은 경우를 선정하여 최종 alpha와 l1_ratio를 선정

    high_ranks = grid_search_result_data_frame[grid_search_result_data_frame['rank_rmse']<=700]
    high_ranks = high_ranks[high_ranks['rank_coef_num']<=50]
    high_ranks = pd.DataFrame(high_ranks)
    print(high_ranks)
    high_ranks_final = high_ranks['rank_rmse'].min()
    for i in range(0,len(high_ranks)):
        if high_ranks.iloc[i,4]==high_ranks_final:
            final_alpha = high_ranks.iloc[i,0]
            final_l1_ratio = high_ranks.iloc[i,1]
            break

    # 최종 변수선택 결과 도출

    selected_coef = elastic_net_feature_selection(x_train,y_train,final_alpha,final_l1_ratio)
    selected_coef.insert(0,'w.1')
    selected_coef.insert(0,'ProdCode')
    selected_coef.insert(0,'Date')
    return selected_coef

In [ ]:
# 0.001간격으로 0부터 1까지의 값인 alpha와 l1_ratio를 각 쌍으로 묶어 그리드서치 진행
# 예 - (alpha,l1_ratio) - (0,0), (0,0.001), ..., (0.001,0), ..., (1,1)

def elastic_net_feature_selection_process(x_train, y_train,alpha,l1_ratio):
    elastic = ElasticNet(alpha=alpha,l1_ratio=l1_ratio)
    elastic.fit(x_train,y_train)

    score = elastic.score(x_train,y_train)
    pred_y = elastic.predict(x_train)
    mse = mean_squared_error(y_train, pred_y)
    rmse = np.sqrt(mse)



    coef = pd.Series(elastic.coef_, index = x_train.columns)
    selected_coef_num = sum(coef != 0)
    return rmse, selected_coef_num

In [ ]:
# 최종 변수선택 결과 도출

def elastic_net_feature_selection(x_train,y_train,alpha,l1_ratio):
    elastic = ElasticNet(alpha=alpha,l1_ratio=l1_ratio)

    elastic.fit(x_train,y_train)

    score = elastic.score(x_train,y_train)
    pred_y = elastic.predict(x_train)
    mse = mean_squared_error(y_train, pred_y)
    rmse = np.sqrt(mse)

    print('RMSE: {0:.4f}'.format(rmse))
    coef = pd.Series(elastic.coef_, index = x_train.columns)
    print("Elastic Net picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
    print(coef[coef !=0])
    coef_selected = coef[coef !=0].index
    coef_selected = list(coef_selected)
    return coef_selected

In [ ]:
k=3
index = 0
# K 결과는 R에서 파악한대로 3으로 고정해서 사용
# 1일 뒤 예측이기에 인덱스는 1로 설정
data_1,data_2,data_3=kmeans_total(data,k,index)
data_1_scale,data_2_scale,data_3_scale = scale_cluster3(data_1,data_2,data_3)
datascale_1,datascale_2,datascale_3=total_scale_cluster3(data_1,data_2,data_3,data_1_scale,data_2_scale,data_3_scale,index)



100%|██████████| 39/39 [00:00<00:00, 569.55it/s]


In [ ]:
selected_coef_1 = elastic_net_cluster(data_1,index)

{'alpha': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'l1_ratio': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 

In [ ]:
selected_coef_2 = elastic_net_cluster(data_2,index)

{'alpha': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'l1_ratio': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 

In [ ]:
selected_coef_3 = elastic_net_cluster(data_3,index)

{'alpha': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'l1_ratio': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 

In [ ]:
data_elasticnet_1 = datascale_1[selected_coef_1]
data_elasticnet_2 = datascale_2[selected_coef_2]
data_elasticnet_3 = datascale_3[selected_coef_3]

In [ ]:
for i in range(1,4):
        if i == 1:
            gaus_result, y_actual_pred = gaus(data_elasticnet_1,i)
        elif i == 2:
            gaus_result2, y_actual_pred2 = gaus(data_elasticnet_2,i)
            gaus_result.update(gaus_result2)
            y_actual_pred.update(y_actual_pred2)

        elif i == 3:
            gaus_result3, y_actual_pred3 = gaus(data_elasticnet_3,i)
            gaus_result.update(gaus_result3)
            y_actual_pred.update(y_actual_pred3)


KeyboardInterrupt: 

In [ ]:
gaus_result

In [ ]:
for i in range(1,4):
    if i == 1:
        RMSE_result = gaus_result[i][0][1]
        MAE_result = gaus_result[i][1][1]
        MAPE_result = gaus_result[i][2][1]
    else :
        RMSE_result = RMSE_result + gaus_result[i][0][1]
        MAE_result = MAE_result + gaus_result[i][1][1]
        MAPE_result = MAPE_result + gaus_result[i][2][1]

RMSE_final_result = RMSE_result/3
MAE_final_result = MAE_result/3
MAPE_final_result = MAPE_result/3

In [ ]:
print(RMSE_final_result)
print(MAE_final_result)
print(MAPE_final_result)